Mount Colab Drive

In [1]:
%cd CoCo
%ls

/media/neurone-pc3/a111f114-c06b-400b-8db7-4ce2466e2265/Andrea/sentence-transformer-proj/CoCo
annotations/                  celegans.graphml.xml  prova/
annotations_trainval2017.zip  cocoapi/              prova_sent_compr/
captions/                     images/               pycocotools.egg-info/


Clone CoCo API repository and run setup/makefile

In [ ]:
!git clone https://github.com/cocodataset/cocoapi.git  # skip this if already downloaded

In [2]:
%cd cocoapi/PythonAPI

/media/neurone-pc3/a111f114-c06b-400b-8db7-4ce2466e2265/Andrea/sentence-transformer-proj/CoCo/cocoapi/PythonAPI


In [ ]:
# Skip this if already downloaded
!ls
!chmod +x Makefile
!make

[Errno 2] No such file or directory: 'cocoapi/PythonAPI'
/content/gdrive/MyDrive/Colab Notebooks/CoCo/cocoapi/PythonAPI
Makefile  pycocoDemo.ipynb  pycocoEvalDemo.ipynb  pycocotools  setup.py
python setup.py build_ext --inplace
running build_ext
cythoning pycocotools/_mask.pyx to pycocotools/_mask.c
/usr/local/lib/python3.10/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/gdrive/MyDrive/Colab Notebooks/CoCo/cocoapi/PythonAPI/pycocotools/_mask.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-cpython-310
creating build/temp.linux-x86_64-cpython-310/pycocotools
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -I/usr/local/li

Change directory again

In [3]:
%cd ../../
!ls

/media/neurone-pc3/a111f114-c06b-400b-8db7-4ce2466e2265/Andrea/sentence-transformer-proj/CoCo
annotations		      celegans.graphml.xml  prova
annotations_trainval2017.zip  cocoapi		    prova_sent_compr
captions		      images		    pycocotools.egg-info


Get CoCo captioning annotations and unzip them

In [ ]:
!wget -c http://images.cocodataset.org/annotations/annotations_trainval2017.zip # skip this if already downloaded

--2023-05-28 14:39:09--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 54.231.199.161, 52.216.236.203, 52.216.248.84, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|54.231.199.161|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

annotations_trainva 100%[===================>] 241.19M  46.2MB/s    in 5.1s    

2023-05-28 14:39:15 (47.5 MB/s) - ‘annotations_trainval2017.zip’ saved [252907541/252907541]



In [ ]:
!unzip -o annotations_trainval2017.zip # skip this if already downloaded

Archive:  annotations_trainval2017.zip
  inflating: annotations/instances_train2017.json  
  inflating: annotations/instances_val2017.json  
  inflating: annotations/captions_train2017.json  
  inflating: annotations/captions_val2017.json  
  inflating: annotations/person_keypoints_train2017.json  
  inflating: annotations/person_keypoints_val2017.json  


Import libraries

In [1]:
import torchvision.datasets as dset
import torchvision.transforms as transforms
import os
!ls
!ls annotations

"ls" non Š riconosciuto come comando interno o esterno,
 un programma eseguibile o un file batch.
"ls" non Š riconosciuto come comando interno o esterno,
 un programma eseguibile o un file batch.


Define custom "CaptionOnly" CoCo dataset returning mock empty images

In [2]:
from typing import Optional, Callable
from PIL import Image
import torch


class CocoCaptionsOnly(dset.CocoCaptions):
  def __init__(
        self,
        root: str,
        ann_file: str,
        transform: Optional[Callable] = None,
        target_transform: Optional[Callable] = None,
        transforms: Optional[Callable] = None,
        dataset_name: str = "CoCoCaptionDataset"
    ) -> None:
        super().__init__(root=root, annFile=ann_file, transforms=transforms,
                         transform=transform, target_transform=target_transform)
        self.__ann_file = ann_file
        self.__dataset_name = dataset_name

  @property
  def ann_file(self) -> str:
    return self.__ann_file

  @property
  def dataset_name(self) -> str:
    return self.__dataset_name

  def _load_image(self, id: int) -> Image.Image:
    # Return mock empty image because we need caption only
    return torch.zeros(3, 427, 640)


In [4]:
cap = CocoCaptionsOnly(root = 'images',
                       dataset_name="coco_caption_val2017",
                       ann_file = 'data/raw/annotations/captions_val2017.json',
                       transform=None)
cap_train = CocoCaptionsOnly(root = 'images',
                             dataset_name="coco_caption_train2017",
                             ann_file = 'data/raw/annotations/captions_train2017.json',
                             transform=None)

print("Number of samples train: ", len(cap_train))
print('Number of samples: ', len(cap))
img, target = cap[3] # load 4th sample

print("Image Size: ", img.size())
print(target)
img, target = cap[4] # load 4th sample

print("Image Size: ", img.size())
print(target)

print(cap.ann_file)
print(cap.dataset_name)

loading annotations into memory...
Done (t=0.15s)
creating index...
index created!
loading annotations into memory...
Done (t=1.03s)
creating index...
index created!
Number of samples train:  118287
Number of samples:  5000
Image Size:  torch.Size([3, 427, 640])
["A stop sign is mounted upside-down on it's post. ", 'A stop sign that is hanging upside down.', 'An upside down stop sign by the road.', 'a stop sign put upside down on a metal pole ', 'A stop sign installed upside down on a street corner']
Image Size:  torch.Size([3, 427, 640])
['Three teddy bears, each a different color, snuggling together.', 'Three stuffed animals are sitting on a bed. ', 'three teddy bears giving each other a hug', 'A group of three stuffed animal teddy bears.', 'Three stuffed bears hugging and sitting on a blue pillow']
data/raw/annotations/captions_val2017.json
coco_caption_val2017


In [5]:
from typing import List, final
import re


MIN_RESPONSE_NUM: final = 5


def mock_llm_response(caption: str, n_responses: int = MIN_RESPONSE_NUM) -> str:
  return "1. Woman wearing a hat;  2. Woman taking a photo;  3. Woman riding " \
         "a bike;  4. Parking lot surrounded by trees;  5. Woman standing in " \
         "the parking lot."


def parse_llm_response(llm_response: str,
                       min_response_num: int = MIN_RESPONSE_NUM) -> List[str]:

  # Split the response on the numbers discarding the first, empty one
  responses = re.split(string=llm_response, pattern=r"[0-9]\.")[1:]

  # Check if the number of created sub-phrases is correct
  assert len(responses) >= min_response_num

  # Format the sub-phrases correctly, removing semi-column, spaces, ...
  for i, response in enumerate(responses):
    responses[i] = response.strip().lower().replace(";", "").replace(".", "")

  return responses


# Test the parsing function
s = mock_llm_response("test0")
s2 = "1. Two monitors sitting on a desk;  2. Desk by a chair;  3. Window;" \
     "4. Monitor and chair;  5. Chair and window."
responses = parse_llm_response(llm_response=s)
responses2 = parse_llm_response(llm_response=s2)
print(responses)
print(responses2)

['woman wearing a hat', 'woman taking a photo', 'woman riding a bike', 'parking lot surrounded by trees', 'woman standing in the parking lot']
['two monitors sitting on a desk', 'desk by a chair', 'window', 'monitor and chair', 'chair and window']


In [8]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.4 MB/s eta 0:00:0031m754.4 kB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
  Using cached transformers-4.30.2-py3-none-any.whl (7.2 MB)
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 47.4 MB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached scipy-1.11.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (36.3 MB)
  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Using cached sentencepiece-0.1.99-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached huggingface_hub-0.15.1-py3-none-any.whl (236 kB)
  Using cached fsspec-2023.6.0-py3-none-any.whl (163 kB)
  Using cached regex-2023.6.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (770 kB)
  Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Using cached safetensors-0.3.1-cp310-cp310-manylinux_2_17_x86_64.ma

Sentence transformer library test

In [6]:
from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer('all-MiniLM-L6-v2')

In [7]:
# Corpus with example sentences
corpus = ['A man is eating food.',
          'A man is eating a piece of bread.',
          'The girl is carrying a baby.',
          'A man is riding a horse.',
          'A woman is playing violin.',
          'Two men pushed carts through the woods.',
          'A man is riding a white horse on an enclosed ground.',
          'A monkey is playing drums.',
          'A cheetah is running behind its prey.'
          ]
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# Query sentences:
queries = ['A man is eating pasta.',
           'Someone in a gorilla costume is playing a set of drums.',
           'A cheetah chases prey on across a field.']


# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(5, len(corpus))
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    print(cos_scores)
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print(corpus[idx], "(Score: {:.4f})".format(score))

    """
    # Alternatively, we can also use util.semantic_search to perform cosine similarty + topk
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=5)
    hits = hits[0]      #Get the hits for the first query
    for hit in hits:
        print(corpus[hit['corpus_id']], "(Score: {:.4f})".format(hit['score']))
    """

tensor([ 0.7035,  0.5272, -0.0898,  0.1889,  0.0336, -0.0594,  0.1047,  0.0819,
         0.0980])




Query: A man is eating pasta.

Top 5 most similar sentences in corpus:
A man is eating food. (Score: 0.7035)
A man is eating a piece of bread. (Score: 0.5272)
A man is riding a horse. (Score: 0.1889)
A man is riding a white horse on an enclosed ground. (Score: 0.1047)
A cheetah is running behind its prey. (Score: 0.0980)
tensor([ 0.0630,  0.0216,  0.0247,  0.1389,  0.2564, -0.0895,  0.1191,  0.6433,
         0.1080])




Query: Someone in a gorilla costume is playing a set of drums.

Top 5 most similar sentences in corpus:
A monkey is playing drums. (Score: 0.6433)
A woman is playing violin. (Score: 0.2564)
A man is riding a horse. (Score: 0.1389)
A man is riding a white horse on an enclosed ground. (Score: 0.1191)
A cheetah is running behind its prey. (Score: 0.1080)
tensor([ 0.1399,  0.0344,  0.0156,  0.0650, -0.0334, -0.0155,  0.1097,  0.1292,
         0.8253])




Query: A cheetah 

Back-translation implementation

In [11]:
!pip install transformers
!pip install openai
!pip install sentencepiece
!pip install sacremoses
!pip install nlpaug
!pip install wordwise
!pip install nltk
!pip install sortedcollections
!pip install datasets

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.4 MB/s eta 0:00:00 MB/s eta 0:00:01
  Using cached aiohttp-3.8.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
  Using cached multidict-6.0.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (114 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB

[nltk_data] Downloading package punkt to /home/neurone-
[nltk_data]     pc3/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/neurone-pc3/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package universal_tagset to /home/neurone-
[nltk_data]     pc3/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

In [8]:
from transformers import MarianMTModel, MarianTokenizer

# Get the name of the first model
first_model_name = 'Helsinki-NLP/opus-mt-en-fr'

# Get the tokenizer
first_model_tkn = MarianTokenizer.from_pretrained(first_model_name)

# Load the pretrained model based on the name
first_model = MarianMTModel.from_pretrained(first_model_name)


# Get the name of the second model
second_model_name = 'Helsinki-NLP/opus-mt-fr-en'

# Get the tokenizer
second_model_tkn = MarianTokenizer.from_pretrained(second_model_name)

# Load the pretrained model based on the name
second_model = MarianMTModel.from_pretrained(second_model_name)

c:\Users\Research\anaconda3\envs\nlp\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Research\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [9]:
original_texts = [
    "This article aims to perform the back translation for text data augmentation",
    "It is the 25th article by Zoumana on Medium. He loves to give back to the community",
    "The first model translates from English to French, which is a temporary process",
    "The second model finally translates back all the temporary french text into English"
]


def format_batch_texts(language_code, batch_texts):

  formated_bach = [">>{}<< {}".format(language_code, text) for text in batch_texts]

  return formated_bach


# Test of the function
formated_batch = format_batch_texts("fr", original_texts)
print(formated_batch)


def perform_translation(batch_texts, model, tokenizer, language="fr"):
    # Prepare the text data into appropriate format for the model
    formated_batch_texts = format_batch_texts(language, batch_texts)

    # Generate translation using model
    translated = model.generate(**tokenizer(formated_batch_texts, return_tensors="pt", padding=True))

    # Convert the generated tokens indices back into text
    translated_texts = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

    return translated_texts

# Check the model translation from the original language (English) to French
translated_texts = perform_translation(original_texts, first_model, first_model_tkn)
print(translated_texts)

# Perform the translation back to English
back_translated_texts = perform_translation(translated_texts, second_model, second_model_tkn)
print(back_translated_texts)


def perform_back_translation(batch_texts,
                             original_language="en",
                             temporary_language="fr"):

  # Translate from Original to Temporary Language
  tmp_translated_batch = perform_translation(batch_texts, first_model,
                                             first_model_tkn,
                                             temporary_language)

  # Translate Back to English
  back_translated_batch = perform_translation(tmp_translated_batch,
                                              second_model,
                                              second_model_tkn,
                                              original_language)

  # Return The Final Result
  return back_translated_batch

# Execute the function for Data Augmentation
final_augmented = perform_back_translation(original_texts)
print(final_augmented)

['>>fr<< This article aims to perform the back translation for text data augmentation', '>>fr<< It is the 25th article by Zoumana on Medium. He loves to give back to the community', '>>fr<< The first model translates from English to French, which is a temporary process', '>>fr<< The second model finally translates back all the temporary french text into English']


c:\Users\Research\anaconda3\envs\nlp\lib\site-packages\transformers\generation\utils.py:1353: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


["Cet article vise à effectuer la traduction arrière pour l'augmentation de données texte", "C'est le 25ème article de Zoumana sur Medium. Il aime rendre à la communauté", "Le premier modèle traduit de l'anglais au français, qui est un processus temporaire", 'Le deuxième modèle traduit enfin tout le texte temporaire français en anglais']
['This article aims to perform back translation for text data increase', "This is Zoumana's 25th article on Medium. He likes to give back to the community.", 'The first model translates from English to French, which is a temporary process', 'The second model finally translates all the temporary French text into English']
['This article aims to perform back translation for text data increase', "This is Zoumana's 25th article on Medium. He likes to give back to the community.", 'The first model translates from English to French, which is a temporary process', 'The second model finally translates all the temporary French text into English']


Positive examples selection

In [10]:
import random
import math
from typing import Union
import nlpaug.augmenter.word as naw


SIMILARITY_THRESHOLD: final = 0.6
N_EXAMPLES: final = 4
TOP_K_BACKUP: final = 2
MAX_ATTEMPTS: final = 5
RANDOM_SEED: final = 14
MAX_WORDS_TO_REPLACE: final = 5
NO_REPLACE_WORDS: final = ["pole", "stop", "sign", "it", "its", "post",
                           "airway", "airways", "a", "the"]


# Set random seed
random.seed(RANDOM_SEED)


# Implement real back translation here
def mock_back_translation(captions: List[str]) -> List[str]:
  return captions


def back_translation(captions: List[str]) -> List[str]:
  return perform_back_translation(captions)


def compute_cosine_scores(caption: str,
                          captions: List[str]) -> torch.Tensor:
  captions_embeddings = embedder.encode(captions, convert_to_tensor=True)
  caption_embedding = embedder.encode(caption, convert_to_tensor=True)
  cos_scores = util.cos_sim(caption_embedding, captions_embeddings)[0]
  return cos_scores


def pop_random_caption(captions: List[str],
                       forbidden: set[str] = frozenset([]),
                       max_iterations: int = 50) -> str:

  if max_iterations == 0:
    raise ValueError(f"max_iterations must be > 0 or < 0, {max_iterations} "
                      "given.")

  caption = None
  iter_counter = 0
  indexes = range(0, len(captions))

  # Until a caption is selected
  while caption is None and iter_counter < max_iterations:
    # Choose a random caption
    i = random.choice(indexes)

    # If it is not in the forbidden list, select it and remove from caption list
    if captions[i] not in forbidden:
      caption = captions[i]
      captions[i], captions[-1] = captions[-1], captions[i]  # O(1) remove
      captions.pop()

    if max_iterations > 0:
      iter_counter += 1

  # If no non-duplicated captions are found in max iter, return a duplicated one
  if caption is None:
    i = random.choice(indexes)
    caption = captions[i]

  return caption


def select_positive_examples(captions: List[str],
                             threshold: float = SIMILARITY_THRESHOLD,
                             n_examples: int = N_EXAMPLES,
                             augment: bool = True,
                             augment_insert: bool = False,
                             return_similarities: bool = False,
                             max_attempts: int = MAX_ATTEMPTS) -> \
                         Union[List[str], tuple[List[str], str, torch.Tensor]]:

  if not 0 < max_attempts <= len(captions):
    raise ValueError(f"max_attempts must be between 1 and the number of "
                     f"captions {len(captions)}. {max_attempts} given.")

  selected_positive_examples = 0  # selected positive example count
  attempt_count = 0  # attempt count
  captions_tmp = []
  already_attempted = set()  # forbidden list to not attempt the same two times

  # Choose a new reference caption and add it to the forbidden list
  caption = pop_random_caption(captions, forbidden=already_attempted)
  already_attempted.add(caption)

  # While we don't select enough positive samples or finish attempts
  while selected_positive_examples == 0 and attempt_count < max_attempts:

    # Compute cosine similarity between the chosen caption and the other ones
    cos_scores = compute_cosine_scores(caption=caption, captions=captions)

    # Discard too distant captions
    captions_tmp = []
    for i, capt in enumerate(captions):
      if cos_scores[i] >= threshold:
        captions_tmp.append(capt)

    # If no captions have been selected
    selected_positive_examples = len(captions_tmp)
    if selected_positive_examples == 0:

      # Increase attempt count
      attempt_count += 1

      # Choose a new reference caption
      caption_new = pop_random_caption(captions, already_attempted)

      # Re-add the previous one to the caption list
      captions.append(caption)

      # Add the new selected caption to the forbidden list (cannot be selected)
      already_attempted.add(caption_new)
      caption = caption_new

  # If no captions have been selected after max attempts, choose the top-k most
  # similar found
  if len(captions_tmp) == 0:
    _, topk_indexes = torch.topk(cos_scores, k=TOP_K_BACKUP)
    for idx in topk_indexes:
      captions_tmp.append(captions[idx])

  captions = captions_tmp

  # (DO NOT) Add the reference caption to the selected positive examples list
  # captions.append(caption)

  # If there aren't enough positive example captions, replicate some and augment
  n_lacking_captions = n_examples - len(captions)
  if n_lacking_captions > 0:

    # If required, perform backtranslation on each positive example caption
    if augment:
      new_captions = back_translation(captions)
      if augment_insert:
        aug = naw.ContextualWordEmbsAug(
          model_path='roberta-base',
          action="insert",
          aug_min=0,
          aug_max=2
        )
        new_captions = aug.augment(new_captions)
      aug = naw.SynonymAug(
        aug_src='wordnet',
        aug_min=1,
        aug_max=MAX_WORDS_TO_REPLACE,
        stopwords=NO_REPLACE_WORDS
      )
      new_captions = aug.augment(new_captions)

    # Otherwise just replicate the existing ones
    else:
      new_captions = captions

    # Add the augmented/replicated captions to the positive example captions,
    # sampling without replacement
    new_captions = random.sample(new_captions,
                                 min(n_lacking_captions, len(new_captions)))

    # If not enough captions were selected, repeat augmented/replicated captions
    if len(new_captions) < n_lacking_captions:
      # current_len*x >= target_len -> x >= target_len/current_len > 1
      n_repeats = math.ceil(n_lacking_captions/len(new_captions)) - 1
      repeated = new_captions*n_repeats
      new_captions.extend(repeated)
      new_captions = new_captions[:n_lacking_captions] # delete excess captions
    captions.extend(new_captions)

  if return_similarities:
    return captions, caption, cos_scores

  return captions


# Test above functions
captions0 = [
    "A stop sign is mounted upside-down on it's post. ",
    'A stop sign that is hanging upside down.',
    'An upside down stop sign by the road.',
    'a stop sign put upside down on a metal pole ',
    'A stop sign installed upside down on a street corner'
]
captions1 = [
    "a british airways jetliner ascending into the clouds",
    "a red white and blue airplane leaving the airport",
    "a plane flying in the air in a cloudy sky.",
    "a plane with its nose raised in a takeoff position.",
    "a white, blue and red jet airliner in sky."
]
captions2 = [
    "a group of people standing around, in the middle of a building.",
    "outside of the baggage claim area of an airport with some passengers waiting and a row of carts stacked with luggage.",
    "two friends stop to talk while others work around them.",
    "some people sitting by a counter at an airport",
    "several baggage trolleys are lined up near loading."
]
captions3 = [
    "two people playing frisbee football with one person outjumping the other to make a catch.",
    "a couple people in a field with a frisbee.",
    "two people playing with a frisbee on a foot ball field",
    "a pair of people playing frisbee against each other on a field.",
    "a pair of people jump for a frisbee."
]


# Test similarity
positive_examples0, caption0, cos_scores0 = select_positive_examples(
    captions=captions0,
    threshold=SIMILARITY_THRESHOLD,
    n_examples=N_EXAMPLES,
    augment=True,
    augment_insert=True,
    return_similarities=True
)
positive_examples1, caption1, cos_scores1 = select_positive_examples(
    captions=captions1,
    threshold=SIMILARITY_THRESHOLD,
    n_examples=N_EXAMPLES,
    augment=True,
    augment_insert=True,
    return_similarities=True
)
positive_examples2, caption2, cos_scores2 = select_positive_examples(
    captions=captions2,
    threshold=SIMILARITY_THRESHOLD,
    n_examples=N_EXAMPLES,
    augment=True,
    augment_insert=True,
    return_similarities=True
)
positive_examples3, caption3, cos_scores3 = select_positive_examples(
    captions=captions3,
    threshold=SIMILARITY_THRESHOLD,
    n_examples=N_EXAMPLES,
    augment=True,
    augment_insert=True,
    return_similarities=True
)

print("Reference example: ", caption0)
print("Positive examples: \n", positive_examples0)
print("Cosine similarities: \n", cos_scores0)
print("Reference example: ", caption1)
print("Positive examples: \n", positive_examples1)
print("Cosine similarities: \n", cos_scores1)
print("Reference example: ", caption2)
print("Positive examples: \n", positive_examples2)
print("Cosine similarities: \n", cos_scores2)
print("Reference example: ", caption3)
print("Positive examples: \n", positive_examples3)
print("Cosine similarities: \n", cos_scores3)

c:\Users\Research\anaconda3\envs\nlp\lib\site-packages\transformers\generation\utils.py:1353: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Research\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Research\AppData\Roaming\nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Research\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


Reference example:  A stop sign is mounted upside-down on it's post. 
Positive examples: 
 ['A stop sign installed upside down on a street corner', 'A stop sign that is hanging upside down.', 'An upside down stop sign by the road.', 'a stop sign put upside down on a metal pole ']
Cosine similarities: 
 tensor([0.7606, 0.8100, 0.7396, 0.7425])
Reference example:  a white, blue and red jet airliner in sky.
Positive examples: 
 ['a british airways jetliner ascending into the clouds', 'a red white and blue airplane leaving the airport', 'a plane flying in the air in a cloudy sky.', 'A crashed British airway jetliner ascending off into the swarm']
Cosine similarities: 
 tensor([0.6360, 0.6991, 0.6571, 0.4684])
Reference example:  several baggage trolleys are lined up near loading.
Positive examples: 
 ['outside of the baggage claim area of an airport with some passengers waiting and a row of carts stacked with luggage.', 'Outside the baggage claim area of a an airport with a few passenger w

In [11]:
# Set OpenAI API key
import openai

api_key = "blablabla"
openai.api_key = api_key

In [12]:
# Setup adaptive crop
from typing import Iterable, List, Optional, final
import math
import random
import re
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
import nlpaug.augmenter.word as naw


MAX_WORDS_TO_REPLACE: final = 6
REPLACE_BERT: final = "replace_bert"
REPLACE_WORDNET: final = "replace_wordnet"
REPLACE_GLOVE: final = "replace_glove"
BACKTRANSL: final = "backtranslation"
NO_REPLACE_WORDS: final = ["pole", "stop", "sign", "it", "its", "post",
                           "airway", "airways", "a", "the", "tree", "trees",
                           "room", "giraffe"]
MAX_INSERT_WORDS: final = 1


def crop_text_based_on_tagging(text: str,
                               crop_prefix: bool = False,
                               max_words_to_cut: Optional[int] = None,
                               augs: Iterable[str] = frozenset([REPLACE_WORDNET,
                                                                BACKTRANSL]),
                               repeat: int = 1,
                               verbose: bool = False) -> List[str]:
  # Cut at most 3/4 of the words
  n_words = len(text.split(" "))
  if max_words_to_cut is None:
    max_words_to_cut = int(4/5 * n_words)
  else:
    max_words_to_cut = min(max_words_to_cut, int(4/5 * n_words))

  new_texts = []
  for _ in range(0, repeat):
    # Choose the words to cut
    n_words_to_cut = random.randint(int(3/4*max_words_to_cut), max_words_to_cut)

    # Get part-of-speech tags
    tags = pos_tag(word_tokenize(text), tagset='universal')
    new_text = text

    if not crop_prefix:
      # Find the last word to keep, it must be either 'VERB' or 'NOUN'
      count_cut_words = 0
      last_word = None
      last_word_idx = None
      for i, el in enumerate(reversed(tags)):
        word, tag = el
        if tag == 'NOUN' or tag == 'VERB':
          last_word = word
          last_word_idx = len(tags) - 1 - i  # reversed index

        # Increase cut word count if the current word is not punctuation
        if tag != ".":
          count_cut_words += 1

        # Stop searching for the last word if we reached the limit
        if count_cut_words >= n_words_to_cut:
          break

      if verbose:
        print(f"Found last word {last_word} on position {last_word_idx}.")

      # Cut all the words after the last one
      new_text = " ".join([word for word, tag in tags[:last_word_idx + 1]])

    else:
      # Find the first word to keep, it must be either 'VERB' or 'NOUN' or 'DET'
      count_cut_words = 0
      first_word = None
      first_word_idx = None
      for i, el in enumerate(tags):
        word, tag = el
        if tag == 'NOUN' or tag == 'VERB' or 'DET':
          first_word = word
          first_word_idx = i  # first word idx

        # Increase cut word count if the current word is not punctuation
        if tag != ".":
          count_cut_words += 1

        # Stop searching for the last word if we reached the limit
        if count_cut_words >= n_words_to_cut:
          break

      if verbose:
        print(f"Found first word {first_word} on position {first_word_idx}.")

      # Cut all the words after the last one
      new_text = " ".join([word for word, tag in tags[first_word_idx:]])

    # Remove spaces before the punctuation
    new_text = re.sub(r'\s([?.!",](?:\s|$))', r'\1', new_text)

    # Apply augmentation
    if BACKTRANSL in augs:
      back_translation_aug = naw.BackTranslationAug(
        from_model_name='Helsinki-NLP/opus-mt-en-fr',
        to_model_name='Helsinki-NLP/opus-mt-fr-en'
      )
      new_text = back_translation_aug.augment(new_text)
    if REPLACE_BERT in augs:
      aug = naw.ContextualWordEmbsAug(
        model_path='roberta-base',
        action="substitute",
        aug_min=1,
        aug_max=2
      )
      new_text = aug.augment(new_text)
    if REPLACE_WORDNET in augs:
      aug = naw.SynonymAug(
        aug_src='wordnet',
        aug_min=1,
        aug_max=MAX_WORDS_TO_REPLACE,
        stopwords=NO_REPLACE_WORDS,
        verbose=True
      )
      new_text = aug.augment(new_text)

    # Add the created text to list
    new_text = new_text[0] if isinstance(new_text, list) else new_text
    new_texts.append(new_text)

  return new_texts


def adaptive_crop_part_pos_examples(caption: str,
                                    n_part_pos_examples: int,
                                    augment_backtranslation: bool = False,
                                    augment_insert: bool = False) -> List[str]:

  part_pos_examples_suffix = crop_text_based_on_tagging(
      text=caption,
      crop_prefix=True,
      max_words_to_cut=None,
      augs=frozenset([REPLACE_WORDNET]),
      repeat=math.ceil(n_part_pos_examples/2)
  )

  part_pos_examples_prefix = crop_text_based_on_tagging(
      text=caption,
      crop_prefix=True,
      max_words_to_cut=None,
      augs=frozenset([REPLACE_WORDNET]),
      repeat=math.floor(n_part_pos_examples/2)
  )
  part_pos_examples = part_pos_examples_suffix + part_pos_examples_prefix

  if augment_backtranslation:
    part_pos_examples = back_translation(part_pos_examples)
  if augment_insert:
    aug = naw.ContextualWordEmbsAug(
      model_path='roberta-base',
      action="insert",
      aug_min=0,
      aug_max=MAX_INSERT_WORDS
    )
    part_pos_examples = aug.augment(part_pos_examples)

  return part_pos_examples


In [30]:
import os
import json
from tqdm import tqdm
import traceback


print(os.getcwd())


CHUNK_DIM: final = 500
N_PART_EXAMPLES: final = 8
CHAT_GPT: final = "chatgpt"
ALPACA: final = "alpaca"
FALCON: final = "falcon"
ADAPTIVE_CROP: final = "adaptive_crop"
ADAPTIVE_CROP_AUGMENT: final = "adaptive_crop_augment"
ID: final = "id"
REFERENCE_EXAMPLE: final = "reference"
POS_EXAMPLES: final = "positive"
NEG_EXAMPLES: final = "negative"
PART_POS_EXAMPLES: final = "part_positive"
CHUNK_NAME: final = "chunk"
ANNOTATION_FILE: final = "ann_file"
INSTANCES: final = "instances"
DATASET_NAME: final = "dataset_name"


def get_falcon_response(caption: str,
                        n_part_pos_examples: int = N_PART_EXAMPLES) -> str:
  raise NotImplementedError("Not implemented yet!")


def get_alpaca_response(caption: str,
                       n_part_pos_examples: int = N_PART_EXAMPLES) -> str:
  raise NotImplementedError("Not implemented yet!")


def get_chatgpt_response(caption: str,
                         n_part_pos_examples: int = N_PART_EXAMPLES) -> str:
  prompt = f"Given the sentence '{caption}' describing a scene, " \
            "identity the main objects/elements and provide 5 very " \
            "short numbered sentences that contain just some " \
            "elements, objects or subjects from sentence and not " \
            "all of them. Do not add any new element, object " \
            "or subject, only use the nouns identified in the given sentence. " \
            "Format the output giving the identified objects and " \
            "the numbered sentences."
  completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[{"role": "user", "content": prompt}],
      temperature=0.1
    )
  response = completion.choices[0].message
  response = response["content"]
  return response



# Implement real LLM interrogation here
def get_part_pos_examples(caption: str,
                          n_part_pos_examples: int = N_PART_EXAMPLES,
                          algorithm_type: str = ADAPTIVE_CROP_AUGMENT) -> List[str]:


  if algorithm_type == CHAT_GPT:
    return parse_llm_response(get_chatgpt_response(caption,
                                                   n_part_pos_examples))
  elif algorithm_type == FALCON:
    return parse_llm_response(get_falcon_response(caption, n_part_pos_examples))

  elif algorithm_type == ALPACA:
    return parse_llm_response(get_alpaca_response(caption, n_part_pos_examples))

  elif algorithm_type == ADAPTIVE_CROP:
    return adaptive_crop_part_pos_examples(caption, n_part_pos_examples)

  elif algorithm_type == ADAPTIVE_CROP_AUGMENT:
    return adaptive_crop_part_pos_examples(caption,
                                           n_part_pos_examples,
                                           augment_insert=False,
                                           augment_backtranslation=True)

  else:
    return parse_llm_response(mock_llm_response(caption, n_part_pos_examples))

def get_chunk_idx(idx: int, chunk_dim: int = CHUNK_DIM) -> tuple[int, int]:
  chunk_idx = math.floor(idx / chunk_dim)  # chunk index
  local_idx = idx % chunk_dim  # local index of the instance inside the chunk
  return chunk_idx, local_idx


def create_dataset_chunck(dataset: CocoCaptionsOnly,
                          start_idx: int = 0,
                          chunk_dim: int = CHUNK_DIM,
                          n_pos_examples: int = N_EXAMPLES,
                          n_part_pos_examples: int = N_PART_EXAMPLES,
                          sim_threshold: float = SIMILARITY_THRESHOLD,
                          augment: bool = True) -> dict:


  end_idx: int = min(start_idx + chunk_dim, len(dataset))  # end index

  dataset_chunk = {
      DATASET_NAME: dataset.dataset_name,
      ANNOTATION_FILE: dataset.ann_file,
      INSTANCES: []
  }
  # For each index in the chunck
  for idx in range(start_idx, end_idx):

    # Get the corresponsing captions
    _, captions = dataset[idx]

    # Get the reference and positive examples
    positive_examples, caption, _ = select_positive_examples(
        captions=captions,
        threshold=sim_threshold,
        n_examples=n_pos_examples,
        augment=augment,
        return_similarities=True,
        max_attempts=n_pos_examples
    )

    # Get the partially positive examples
    part_pos_examples = get_part_pos_examples(
        caption=caption,
        n_part_pos_examples=n_part_pos_examples,
        algorithm_type=ADAPTIVE_CROP
    )

    # Add dataset entry to chuck
    dataset_entry = {
        ID: idx,
        REFERENCE_EXAMPLE: caption,
        POS_EXAMPLES: positive_examples,
        PART_POS_EXAMPLES: part_pos_examples
    }
    dataset_chunk[INSTANCES].append(dataset_entry)

  return dataset_chunk


def create_dataset(root: str,
                   dataset: CocoCaptionsOnly,
                   start_chunk: int = 0,
                   max_chunk: Optional[int] = None,
                   chunk_dim: int = CHUNK_DIM,
                   n_pos_examples: int = N_EXAMPLES,
                   n_part_pos_examples: int = N_PART_EXAMPLES,
                   sim_threshold: float = SIMILARITY_THRESHOLD,
                   augment: bool = True,
                   log_tqdm: bool = False) -> int:
  # Create dataset directory if it doesn't exist
  root = os.path.join(root, dataset.dataset_name)
  os.makedirs(root, exist_ok=True)

  # Calculate max chunk
  chunk_num = math.ceil(len(dataset) / chunk_dim)
  chunk_num = chunk_num if max_chunk is None else min(chunk_num, max_chunk)

  # For each chunk
  iterable = range(start_chunk, chunk_num)
  iterable = tqdm(iterable, "Creating chunks...") if log_tqdm else iterable
  last_chunk_idx = -1
  for chunk_idx in iterable:

    try:
      # Create the chunk
      dataset_chunk = create_dataset_chunck(
          dataset=dataset,
          start_idx=chunk_idx*chunk_dim,
          n_pos_examples=n_pos_examples,
          n_part_pos_examples=n_part_pos_examples,
          sim_threshold=sim_threshold,
          augment=augment
      )

      # Store the chunk
      chunk_filename = os.path.join(root, f"{CHUNK_NAME}_{chunk_idx}.json")
      with open(chunk_filename, "w") as fp:
        json.dump(dataset_chunk, fp)
      last_chunk_idx = chunk_idx

    except Exception as e:
      print(f"Chunk {chunk_idx} creation failed due to error: {e}.")
      print(f"Traceback: {traceback.format_exc()}")
      return chunk_idx - 1

  return last_chunk_idx


C:\Users\Research\Desktop\quadruplet-sentence-transformer


In [31]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Research\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Research\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\Research\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [30]:
print(os.getcwd())
%cd ..
print(os.getcwd())

/media/neurone-pc3/a111f114-c06b-400b-8db7-4ce2466e2265/Andrea/sentence-transformer-proj/CoCo
/media/neurone-pc3/a111f114-c06b-400b-8db7-4ce2466e2265/Andrea/sentence-transformer-proj
/media/neurone-pc3/a111f114-c06b-400b-8db7-4ce2466e2265/Andrea/sentence-transformer-proj


In [33]:
print(CHUNK_DIM)
create_dataset("./data/coco/train", dataset=cap_train, max_chunk=None, log_tqdm=True)  # skip this if already created

500


Creating chunks...:   0%|          | 1/237 [51:51<203:57:18, 3111.18s/it]

[Warning] Name:Out of vocabulary issue, Code:W002, Message:No other word except stop words and OOV. Returning input data without augmentation
[Warning] Name:Out of vocabulary issue, Code:W002, Message:No other word except stop words and OOV. Returning input data without augmentation
[Warning] Name:Out of vocabulary issue, Code:W002, Message:No other word except stop words and OOV. Returning input data without augmentation
[Warning] Name:Out of vocabulary issue, Code:W002, Message:No other word except stop words and OOV. Returning input data without augmentation


Creating chunks...:   1%|          | 2/237 [1:41:28<197:56:19, 3032.25s/it]

[Warning] Name:Out of vocabulary issue, Code:W002, Message:No other word except stop words and OOV. Returning input data without augmentation
[Warning] Name:Out of vocabulary issue, Code:W002, Message:No other word except stop words and OOV. Returning input data without augmentation
[Warning] Name:Out of vocabulary issue, Code:W002, Message:No other word except stop words and OOV. Returning input data without augmentation
[Warning] Name:Out of vocabulary issue, Code:W002, Message:No other word except stop words and OOV. Returning input data without augmentation
[Warning] Name:Out of vocabulary issue, Code:W002, Message:No other word except stop words and OOV. Returning input data without augmentation
[Warning] Name:Out of vocabulary issue, Code:W002, Message:No other word except stop words and OOV. Returning input data without augmentation


...

[Warning] Name:Out of vocabulary issue, Code:W002, Message:No other word except stop words and OOV. Returning input data without augmentation
[Warning] Name:Out of vocabulary issue, Code:W002, Message:No other word except stop words and OOV. Returning input data without augmentation
[Warning] Name:Out of vocabulary issue, Code:W002, Message:No other word except stop words and OOV. Returning input data without augmentation


Creating chunks...:   4%|▍         | 10/237 [8:14:02<186:29:19, 2957.53s/it]

[Warning] Name:Out of vocabulary issue, Code:W002, Message:No other word except stop words and OOV. Returning input data without augmentation
[Warning] Name:Out of vocabulary issue, Code:W002, Message:No other word except stop words and OOV. Returning input data without augmentation
[Warning] Name:Out of vocabulary issue, Code:W002, Message:No other word except stop words and OOV. Returning input data without augmentation


Creating chunks...:   5%|▍         | 11/237 [9:05:21<186:44:37, 2974.68s/it]


KeyboardInterrupt: 

In [32]:
print(CHUNK_DIM)
create_dataset("./data/coco/validation", dataset=cap, max_chunk=None, log_tqdm=True)  # skip this if already created

500


Creating chunks...:   0%|          | 0/10 [00:00<?, ?it/s]c:\Users\Research\anaconda3\envs\nlp\lib\site-packages\transformers\generation\utils.py:1353: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Creating chunks...:  10%|█         | 1/10 [49:14<7:23:12, 2954.73s/it]

[Warning] Name:Out of vocabulary issue, Code:W002, Message:No other word except stop words and OOV. Returning input data without augmentation
[Warning] Name:Out of vocabulary issue, Code:W002, Message:No other word except stop words and OOV. Returning input data without augmentation
[Warning] Name:Out of vocabulary issue, Code:W002, Message:No other word except stop words and OOV. Returning input data without augmentation
[Warning] Name:Out of vocabulary issue, Code:W002, Message:No other word except stop words and OOV. Returning input data without augmentation
[Warning] Name:Out of vocabulary issue, Code:W002, Message:No other word except stop words and OOV. Returning input data without augmentation
[Warning] Name:Out of vocabulary issue, Code:W002, Message:No other word except stop words and OOV. Returning input data without augmentation
[Warning] Name:Out of vocabulary issue, Code:W002, Message:No other word except stop words and OOV. Returning input data without augmentation
[Warni

Creating chunks...:  20%|██        | 2/10 [1:35:55<6:21:53, 2864.20s/it]

[Warning] Name:Out of vocabulary issue, Code:W002, Message:No other word except stop words and OOV. Returning input data without augmentation


Creating chunks...:  40%|████      | 4/10 [3:08:35<4:42:09, 2821.52s/it]

[Warning] Name:Out of vocabulary issue, Code:W002, Message:No other word except stop words and OOV. Returning input data without augmentation
[Warning] Name:Out of vocabulary issue, Code:W002, Message:No other word except stop words and OOV. Returning input data without augmentation
[Warning] Name:Out of vocabulary issue, Code:W002, Message:No other word except stop words and OOV. Returning input data without augmentation
[Warning] Name:Out of vocabulary issue, Code:W002, Message:No other word except stop words and OOV. Returning input data without augmentation
[Warning] Name:Out of vocabulary issue, Code:W002, Message:No other word except stop words and OOV. Returning input data without augmentation
[Warning] Name:Out of vocabulary issue, Code:W002, Message:No other word except stop words and OOV. Returning input data without augmentation


Creating chunks...: 100%|██████████| 10/10 [8:06:31<00:00, 2919.14s/it] 


9

Create dataset for dataloader ---QUI---

In [27]:
from typing import Callable, List, Union, final, Dict
import json
import time
import heapq
import torch
import numpy as np
from sortedcollections import ItemSortedDict, ValueSortedDict


CACHE_SIZE_DEFAULT: final = 30
HARD_CONTRASTIVE_TRAIN: final = 1
HARD_CONTRASTIVE_TEST: final = 0
RANDOM: final = -1
NEG_EXAMPLE_SIM_TRESHOLD: final = 0.2
MAX_ATTEMPTS_NEGATIVE_SAMPLING: final = 3
_CACHE_LOG: final = False


def hard_constrastive_sampling(population: List,
                               scores: Union[np.ndarray, List[float]],
                               k: int,
                               max_mode: bool = True) -> List:
  # TODO: test if it's correct checking scores of the outputed captions
  if max_mode:
    key = lambda idx, score: score  # should select maximum
  else:
    key = lambda idx, score: -score  # should select minimum

  # Select the top-k largest scores
  scores = [(i, score) for i, score in enumerate(scores)]
  scores = heapq.nlargest(k, scores, key=key)

  # Select the corresponding objects
  selected = [population[i] for i, score in scores]

  return selected

def _choose_examples(choices: List[str], n: int) -> List[str]:
  examples = set()
  for _ in range(0, n):
    example = pop_random_caption(captions=choices, forbidden=examples)
    examples.add(example)

  return list(examples)


class QuadrupletDataset(torch.utils.data.Dataset):
  def __init__(self,
               root: str,
               *dataset_chunks: str,
               hard_contrastive_mode: int = RANDOM,
               n_pos: int = 1,
               n_part_pos: int = 1,
               n_neg: int = 1,
               cache_size: int = CACHE_SIZE_DEFAULT,
               transform: Callable = None):
    if cache_size <= 0:
      raise ValueError(f"Cache size must be > 0. {cache_size} given.")
    if len(dataset_chunks) == 0:
      raise ValueError(f"At least 1 chunk must be given.")
    if not -1 <= hard_contrastive_mode <= 1:
      raise ValueError(f"hard_contrastive_mode must be either {RANDOM}, "
                       f"{HARD_CONTRASTIVE_TRAIN} or {HARD_CONTRASTIVE_TEST}, "
                       f"respectively meaning no hard contrastive, hard "
                       f"contrastive training mode and hard contrastive test "
                       f"mode. {hard_contrastive_mode} given.")
    if n_pos <= 0:
      raise ValueError(f"n_pos must be > 0, {n_pos} given.")
    if n_part_pos <= 0:
      raise ValueError(f"n_part_pos must be > 0, {n_part_pos} given.")
    if n_neg <= 0:
      raise ValueError(f"n_neg must be > 0, {n_neg} given.")

    self.__root: str = root
    self.__hard_contrastive_mode: int = hard_contrastive_mode
    self.__n_pos: int = n_pos
    self.__n_part_pos: int = n_part_pos
    self.__n_neg: int = n_neg
    self._transform: Callable = transform
    self.__dataset_chunks: List[str] = [*dataset_chunks]

    # Ordered set that stores couples (chunk, last_access_timestamp), ordered by
    # last_acces_timestamp
    self.__chunk_cache = ValueSortedDict(lambda value: value[1])
    self.__cache_size: int = cache_size

    # Compute chunk dim (assuming all chunks have the same dim, except the last)
    chunk_path = os.path.join(root, self.__dataset_chunks[0])
    with open(chunk_path) as fp:
      chunk = json.load(fp)
      self.__chunk_dim = len(chunk[INSTANCES])

    # Compute last chunk dim
    chunk_path = os.path.join(root, self.__dataset_chunks[-1])
    with open(chunk_path) as fp:
      chunk = json.load(fp)
      self.__last_chunk_dim = len(chunk[INSTANCES])

  @property
  def root(self) -> str:
    return self.__root

  @property
  def hard_contrastive_mode(self) -> int:
    return self.__hard_contrastive_mode

  @hard_contrastive_mode.setter
  def hard_contrastive_mode(self, hard_contrastive_mode: int):
    if not -1 <= hard_contrastive_mode <= 1:
      raise ValueError(f"hard_contrastive_mode must be either {RANDOM}, "
                       f"{HARD_CONTRASTIVE_TRAIN} or {HARD_CONTRASTIVE_TEST}, "
                       f"respectively meaning no hard contrastive, hard "
                       f"contrastive training mode and hard contrastive test "
                       f"mode. {hard_contrastive_mode} given.")
    self.__hard_contrastive_mode = hard_contrastive_mode

  @property
  def n_pos(self) -> int:
    return self.__n_pos

  @n_pos.setter
  def n_pos(self, n_pos: int):
    if n_pos <= 0:
      raise ValueError(f"n_pos must be > 0, {n_pos} given.")
    self.__n_pos = n_pos

  @property
  def n_part_pos(self) -> int:
    return self.__n_part_pos

  @n_part_pos.setter
  def n_part_pos(self, n_part_pos: int):
    if n_part_pos <= 0:
      raise ValueError(f"n_part_pos must be > 0, {n_part_pos} given.")
    self.__n_part_pos = n_part_pos

  @property
  def n_neg(self) -> int:
    return self.__n_neg

  @n_neg.setter
  def n_neg(self, n_neg: int):
    if n_neg <= 0:
      raise ValueError(f"n_neg must be > 0, {n_neg} given.")
    self.__n_neg = n_neg

  @property
  def dataset_chunks(self) -> List[str]:
    return self.__dataset_chunks

  @property
  def n_chunks(self) -> int:
    return len(self.dataset_chunks)

  @property
  def chunk_dim(self) -> int:
    return self.__chunk_dim

  @property
  def last_chunk_dim(self) -> int:
    return self.__last_chunk_dim

  @property
  def cache_size(self) -> int:
    return self.__cache_size

  def __len__(self) -> int:
    return (self.n_chunks - 1)*self.chunk_dim + self.last_chunk_dim


  def choose_negative(self,
                      chunk_idx: int,
                      instance: Dict,
                      local_idx: int,
                      n: int,
                      hard_contrastive_mode: int = RANDOM) -> List[str]:
    if not -1 <= hard_contrastive_mode <= 1:
      raise ValueError(f"hard_contrastive_mode must be either {RANDOM}, "
                        f"{HARD_CONTRASTIVE_TRAIN} or {HARD_CONTRASTIVE_TEST}, "
                        f"respectively meaning no hard contrastive, hard "
                        f"contrastive training mode and hard contrastive test "
                        f"mode. {hard_contrastive_mode} given.")

    # Select a random chunk to sample from
    selected_chunk_idx = random.randint(0, self.n_chunks - 1)
    chunk, _, _ = self.__get_chunk(selected_chunk_idx*self.chunk_dim)
    instance_indexes = list(range(len(chunk[INSTANCES])))

    # If the selected chunk is the one the current instance is coming, remove it
    if selected_chunk_idx == chunk_idx:
      instance_indexes[local_idx], instance_indexes[-1] = instance_indexes[-1], instance_indexes[local_idx]  # exchange local_idx and last
      instance_indexes.pop()  # remove the last (now local_idx)

    diff = -1
    attempt_count = 0
    selected_captions_total = []
    # Continue to sample caption until we have n
    while diff < 0 and attempt_count < MAX_ATTEMPTS_NEGATIVE_SAMPLING:

      # Choose at most n*5 instances to select the captions from
      selected_instance_indexes = random.sample(
          population=instance_indexes,
          k=min(math.ceil(n*5), len(instance_indexes))
      )

      selected_captions = []
      for i in selected_instance_indexes:
        caption = random.choice(chunk[INSTANCES][i][POS_EXAMPLES])
        selected_captions.append(caption)
      selected_captions = np.array(selected_captions)

      # Discard captions too similar to the reference instance, according to SBERT
      reference_caption = instance[REFERENCE_EXAMPLE]
      cos_scores = compute_cosine_scores(
          caption=reference_caption,
          captions=selected_captions
      ).detach().cpu().numpy()
      selected_captions = selected_captions[cos_scores <= NEG_EXAMPLE_SIM_TRESHOLD]
      cos_scores = cos_scores[cos_scores <= NEG_EXAMPLE_SIM_TRESHOLD]
      selected_captions = selected_captions.tolist()
      selected_captions_total.extend(selected_captions)

      # Among the chosen <= n*2, select n negative, either randomly or with HCS
      attempt_count += 1
      diff = len(selected_captions_total) - n

    if diff > 0:
      if hard_contrastive_mode == HARD_CONTRASTIVE_TRAIN:
        # Select the hardest
        selected_captions_total = hard_constrastive_sampling(
            population=selected_captions_total,
            scores=cos_scores,
            max_mode=True,
            k=n
        )
      else:
        # Random sampling
        selected_captions_total = random.sample(selected_captions_total, k=n)
      """
      elif hard_contrastive_mode == HARD_CONTRASTIVE_TEST:
        # Select the easiest, or maybe random
        selected_captions = hard_constrastive_sampling(
            population=selected_captions,
            scores=score,
            max_mode=True,
            k=n
        )
      """

    # Replicate the selected until we have n negative examples
    if diff < 0:
      replicated = random.choices(selected_captions_total, k=abs(diff))
      selected_captions_total.extend(replicated)

    return selected_captions_total

  def example_sampling(self,
                        instance: Dict,
                        local_idx: int,
                        chunk_idx: int,
                        n_pos: int = 1,
                        n_part_pos: int = 1,
                        n_neg: int = 1,
                        hard_contrastive_mode: int = RANDOM) -> Dict:
    if n_pos <= 0:
      raise ValueError(f"n_pos must be > 0, {n_pos} given.")
    if n_part_pos <= 0:
      raise ValueError(f"n_part_pos must be > 0, {n_part_pos} given.")
    if n_neg <= 0:
      raise ValueError(f"n_neg must be > 0, {n_neg} given.")
    if not -1 <= hard_contrastive_mode <= 1:
      raise ValueError(f"hard_contrastive_mode must be either {RANDOM}, "
                        f"{HARD_CONTRASTIVE_TRAIN} or {HARD_CONTRASTIVE_TEST}, "
                        f"respectively meaning no hard contrastive, hard "
                        f"contrastive training mode and hard contrastive test "
                        f"mode. {hard_contrastive_mode} given.")


    # Positive examples
    choices = list(instance[POS_EXAMPLES])
    pos_examples: list[str] = _choose_examples(choices=choices, n=n_pos)
    if len(pos_examples) == 1:
      pos_examples = pos_examples[0]

    # Partially positive examples
    choices = list(instance[PART_POS_EXAMPLES])
    part_pos_examples: list[str] = _choose_examples(choices=choices,
                                                    n=n_part_pos)
    if len(part_pos_examples) == 1:
      part_pos_examples = part_pos_examples[0]

    # Negative examples, choose candidate negative examples from random chunk
    neg_examples = self.choose_negative(
        chunk_idx=chunk_idx,
        instance=instance,
        local_idx=local_idx,
        n=n_neg,
        hard_contrastive_mode=hard_contrastive_mode
    )

    if len(neg_examples) == 1:
      neg_examples = neg_examples[0]

    return_instance = instance.copy()
    return_instance[POS_EXAMPLES] = pos_examples
    return_instance[PART_POS_EXAMPLES] = part_pos_examples
    return_instance[NEG_EXAMPLES] = neg_examples

    return return_instance


  def __get_chunk(self, idx) -> tuple[Dict, int, int]:
    """
    Gets the chunk in which the required item resides, handling the caching.

    :param idx: the index of the required item.

    :return the chunk in which the required item resides and the index of the
      item inside the chunk.
    """
    # Get chunk index
    chunk_idx, local_idx = get_chunk_idx(idx, chunk_dim=self.chunk_dim)

    if _CACHE_LOG:
      print(f"Accessing chunk {chunk_idx}")

    # If chunk is cached
    if chunk_idx in self.__chunk_cache:

      # Get it from the cache
      chunk = self.__chunk_cache[chunk_idx][0]

      # Refresh its value, putting it at the top
      self.__chunk_cache[chunk_idx] = [chunk, time.time()] # O(logn)

    # If chunk is not cached
    else:
      # Read it from the disk and add it to the cache
      chunk_path = os.path.join(self.root, self.__dataset_chunks[chunk_idx])
      with open(chunk_path, "r") as fp:
        chunk = json.load(fp)

      # Check if cache is full, and in that case, delete the oldest item
      if len(self.__chunk_cache) == self.__cache_size:

        if _CACHE_LOG:
          print([(k, self.__chunk_cache[k][1])  for k in self.__chunk_cache.keys()])

        item = self.__chunk_cache.popitem(index=0)  # O(logn)

        if _CACHE_LOG:
          print(f"Deleted chunk: {item[0]} with value {item[1][1]}")

      # Add the read chunk to the cache
      self.__chunk_cache[chunk_idx] = [chunk, time.time()]  # O(logn)

    return chunk, local_idx, chunk_idx


  def __getitem__(self, idx) -> Union[Dict[str, str], List[Dict[str, str]]]:
    if torch.is_tensor(idx):
      idx = idx.tolist()
    if not isinstance(idx, list):
      idx = [idx]

    instances = []
    # For each index
    for i in idx:
      # Get the corresponsing chunk and instance
      chunk, local_idx, chunk_idx = self.__get_chunk(i)
      instance = chunk[INSTANCES][local_idx]

      # Hard contrastive/random sample the neg., sample pos. and part. pos.
      instance = self.example_sampling(
          instance=instance,
          local_idx=local_idx,
          chunk_idx=chunk_idx,
          hard_contrastive_mode=self.hard_contrastive_mode,
          n_pos=self.n_pos,
          n_part_pos=self.n_part_pos,
          n_neg=self.n_neg
      )

      # Apply transform
      if self._transform is not None:
        instance = self._transform(instance)

      # Add to the instances
      instances.append(instance)

    return instances if len(instances) > 1 else instances[0]


In [29]:
from torch.utils.data import DataLoader

print(os.getcwd())

qds = QuadrupletDataset("./data/coco/train/coco_caption_train2017",
                        *[f"chunk_{i}.json" for i in range(0, 10)],
                        hard_contrastive_mode = RANDOM,
                        n_pos=1,
                        n_neg=1,
                        n_part_pos=1,
                        cache_size=3)

# 1st chunk
print(qds[0])
print(qds[3])
print(qds[4])

# 2nd chunk
print(qds[qds.chunk_dim + 2])
print(qds[qds.chunk_dim + 4])

# 3rd chunk
print(qds[qds.chunk_dim*2])
print(qds[qds.chunk_dim*2 + 3])

# 5th chunk
print(qds[qds.chunk_dim*4 + 1])
print(qds[qds.chunk_dim*4 + 5])

# 6th chunk
print(qds[qds.chunk_dim*5 + 2])
print(qds[qds.chunk_dim*5 + 9])

# 3rd chunk
print(qds[qds.chunk_dim*2 + 2])
print(qds[qds.chunk_dim*2 + 2])

# 1st chunk
print(qds[6])
print(qds[2])

# Get more than one sample
print(qds[[1, 2, 3]])

#train_dataloader = DataLoader(training_data, batch_size=2, shuffle=True)

C:\Users\Research\Desktop\quadruplet-sentence-transformer
{'id': 0, 'reference': 'there are containers filled with different kinds of foods', 'positive': 'A mountain of trays that throw different food for thought.', 'part_positive': 'kinds of foods', 'negative': 'Two giraffes in a room with people looking at them.'}
{'id': 3, 'reference': 'A lone zebra grazing in some green grass.', 'positive': 'a Zebra grazing on grass in a green open field.', 'part_positive': 'some immature pasture.', 'negative': 'Masses flying kite in a champaign on a cloudy day.'}
{'id': 4, 'reference': 'A woman in a floral swimsuit holds a pink umbrella.', 'positive': 'A womanhood posing for the photographic camera, holding a pink umbrella, open and wearing a bright, floral swimming costume, beehiveed, by a lifeguard stand with lake, unripe trees, and a blue sky with a few clouds behind.', 'part_positive': 'a garden pink umbrella.', 'negative': 'there are people riding elephants in the middle of a forest'}
{'id': 

In [35]:
from torch.utils.data import DataLoader

print(os.getcwd())

qds = QuadrupletDataset("./data/coco/coco_caption_val2017",
                        *[f"chunk_{i}.json" for i in range(0, 10)],
                        hard_contrastive_mode = RANDOM,
                        n_pos=3,
                        n_neg=5,
                        n_part_pos=2,
                        cache_size=7)

# 1st chunk
print(qds[0])
print(qds[3])
print(qds[4])

# 2nd chunk
print(qds[qds.chunk_dim + 2])
print(qds[qds.chunk_dim + 4])

# 3rd chunk
print(qds[qds.chunk_dim*2])
print(qds[qds.chunk_dim*2 + 3])

# 5th chunk
print(qds[qds.chunk_dim*4 + 1])
print(qds[qds.chunk_dim*4 + 5])

# 6th chunk
print(qds[qds.chunk_dim*5 + 2])
print(qds[qds.chunk_dim*5 + 9])

# 3rd chunk
print(qds[qds.chunk_dim*2 + 2])
print(qds[qds.chunk_dim*2 + 3])

# 1st chunk
print(qds[6])
print(qds[2])


# Get more than one sample
print(qds[[1, 2, 3]])
#train_dataloader = DataLoader(training_data, batch_size=2, shuffle=True)

/media/neurone-pc3/a111f114-c06b-400b-8db7-4ce2466e2265/Andrea/sentence-transformer-proj
{'id': 0, 'reference': 'A person standing at a table in a room.', 'positive': ['A woman stands in the dining area at the table.', 'A room with chairs, a table and a adult female in it.', 'A room with chairs, a table, and a woman in it.'], 'part_positive': ['a mesa in a room.', 'a tabular array in a room.'], 'negative': ['An elephant using his truck to cuddle with a man.', 'a bunch of kids sitting on grass holding a frisbe', 'A man who disembarrass a undulation on a surfboard.', 'A woman in a costume speaks on a cellphone. ', 'A little girl holding an umbrella in the rain. ']}
{'id': 3, 'reference': "A stop sign is mounted upside-down on it's post. ", 'positive': ['a stop sign put upside down on a metal pole ', 'A stop sign that is hanging upside down.', 'A stop sign installed upside down on a street corner'], 'part_positive': ["it ' s post.", "upside - downwards on it ' s post."], 'negative': ['A s

In [21]:
from datasets import load_dataset


# Second type of dataset
sentence_compression_ds = load_dataset("sent_comp")



print(sentence_compression_ds["train"][1]["graph"]["sentence"])
print(sentence_compression_ds["train"][1]["compression"]["text"])
print(sentence_compression_ds["train"][1]["compression_ratio"])

print(sentence_compression_ds["train"][2]["graph"]["sentence"])
print(sentence_compression_ds["train"][2]["compression"]["text"])
print(sentence_compression_ds["train"][2]["compression_ratio"])

print(sentence_compression_ds["validation"][2]["graph"]["sentence"])
print(sentence_compression_ds["validation"][2]["compression"]["text"])
print(sentence_compression_ds["validation"][2]["compression_ratio"])

print(sentence_compression_ds["validation"][3]["graph"]["sentence"])
print(sentence_compression_ds["validation"][3]["compression"]["text"])
print(sentence_compression_ds["validation"][3]["compression_ratio"])

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/200000 [00:00<?, ? examples/s]

Dataset sent_comp downloaded and prepared to /home/neurone-pc3/.cache/huggingface/datasets/sent_comp/default/0.0.0/512501fef5db888ec620cb9e4943420ea7c7c244c60de9222fb50bca1232f4b5. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

MILAN -Catania held Roma to a 1-1 draw in Serie A on Wednesday as the teams played out the remaining 25 minutes of a game that was called off last month.
Catania held Roma to a 1 1 draw in Serie A.
0.2810457646846771
State Street Corporation, a provider of investment servicing, investment management and investment research and trading services, has launched a new investment servicing solution to support small to mid-sized asset managers with their investment operations needs.
State Street Corporation, has launched a new investment servicing solution.
0.2851710915565491
Luis Suarez was spotted in London this afternoon and this has led the Daily Star to link the Liverpool striker to a potential move to Chelsea or Arsenal.
Luis Suarez was spotted in London.
0.2222222238779068
A woman was injured by a falling tree in the Gresham neighborhood, according to the Chicago Fire Department.
A woman was injured by a falling tree.
0.35185185074806213


In [22]:
# Get dataset length
print("Train dim: ", len(sentence_compression_ds["train"]))
print("Val dim: ", len(sentence_compression_ds["validation"]))
print("Train chunks: ", math.ceil(len(sentence_compression_ds["train"]) / 1000))
print("Val chunks: ", math.ceil(len(sentence_compression_ds["validation"]) / 1000))

Train dim:  200000
Val dim:  10000
Train chunks:  200
Val chunks:  10


In [23]:
import numpy as np
import traceback


TRAIN_SENT_COMPR: final = "train"
TEST_SENT_COMPR: final = "validation"
GRAPH: final = "graph"
COMPRESSION: final = "compression"
COMPR_RATIO: final = "compression_ratio"
SENTENCE: final = "sentence"
TEXT: final = "text"
COMPRESSION_RATIO_TRESHOLD: final = 0.6

INSERT_BERT: final = "insert_bert"
MAX_INSERT_BERT: final = 4
MAX_REPLACE_BERT: final = 3
DEFAULT_AUGS: final = frozenset([REPLACE_WORDNET, BACKTRANSL, INSERT_BERT])


def generate_variations(sentence: Union[str, List[str]],
                        n: int,
                        augs: Iterable[str] = DEFAULT_AUGS) -> List[str]:
  sentences = list(np.repeat(sentence, n))

  if INSERT_BERT in augs:
    aug = naw.ContextualWordEmbsAug(
      model_path='roberta-base',
      action="insert",
      aug_min=0,
      aug_max=MAX_INSERT_BERT
    )
    sentences = aug.augment(sentences)
  if REPLACE_BERT in augs:
    aug = naw.ContextualWordEmbsAug(
      model_path='roberta-base',
      action="substitute",
      aug_min=1,
      aug_max=MAX_REPLACE_BERT
    )
    sentences = aug.augment(sentences)
  if REPLACE_WORDNET in augs:
    aug = naw.SynonymAug(
      aug_src='wordnet',
      aug_min=1,
      aug_max=4,
      stopwords=NO_REPLACE_WORDS,
      verbose=True
    )
    sentences = aug.augment(sentences)
  if BACKTRANSL in augs:
      sentences = back_translation(sentences)
  return sentences

print(generate_variations("A woman wearing a strange hat is taking a photo, sitting on her bike in a parking lot surrounded by trees.", n=5))



def get_pos_examples_sentence_compr(sentence_compression_instance: Dict,
                                    n: int) -> List[str]:
  reference_text = sentence_compression_instance[GRAPH][SENTENCE]
  compression = sentence_compression_instance[COMPRESSION][TEXT]
  compression_ratio = sentence_compression_instance[COMPR_RATIO]
  pos_examples = [reference_text]

  # If compression ratio is small, consider the compressed as positive example
  if compression_ratio >= COMPRESSION_RATIO_TRESHOLD:
    pos_examples.append(compression)

  # Generate variations and add it to the pos_examples
  n = n - 1 if len(pos_examples) == 2 else n
  variations = generate_variations(reference_text, n=n)
  pos_examples.extend(variations)

  return pos_examples, reference_text


def get_part_pos_examples_sentence_compr(sentence_compression_instance: Dict,
                                         n: int) -> List[str]:
  reference_text = sentence_compression_instance[GRAPH][SENTENCE]
  compression = sentence_compression_instance[COMPRESSION][TEXT]
  compression_ratio = sentence_compression_instance[COMPR_RATIO]

  part_pos_examples = []
  # If compression ratio is high, consider the compressed as part pos example
  if compression_ratio < COMPRESSION_RATIO_TRESHOLD:
    part_pos_examples.append(compression)

    # Generate variations of the part pos examples
    variations = generate_variations(reference_text,
                                     n=math.ceil(n/2),
                                     augs=[REPLACE_WORDNET, BACKTRANSL])
    part_pos_examples.extend(variations)
    n = math.floor(n/2) - 1

  # Get the remaining partially positive examples with adaptive crop
  adaptive_crop_examples = get_part_pos_examples(
      caption=reference_text,
      n_part_pos_examples=n,
      algorithm_type=ADAPTIVE_CROP_AUGMENT
  )
  part_pos_examples.extend(adaptive_crop_examples)

  return part_pos_examples


def create_dataset_chunck_sentence_compression(dataset,
                                               dataset_name: str = "sent_compr",
                                               split: str = TRAIN_SENT_COMPR,
                                               start_idx: int = 0,
                                               chunk_dim: int = CHUNK_DIM,
                                               n_pos_examples: int = N_EXAMPLES,
                                               n_part_pos_examples: int = N_PART_EXAMPLES) -> dict:


  end_idx: int = min(start_idx + chunk_dim, len(dataset[split]))  # end index

  dataset_chunk = {
      DATASET_NAME: dataset_name,
      ANNOTATION_FILE: None,
      INSTANCES: []
  }
  # For each index in the chunck
  for idx in range(start_idx, end_idx):

    # Get the corresponsing instance
    sentence_compression_instance = dataset[split][idx]

    # Get the reference and positive examples
    positive_examples, caption = get_pos_examples_sentence_compr(
        sentence_compression_instance,
        n=n_pos_examples
    )

    # Get the partially positive examples
    part_pos_examples = get_part_pos_examples_sentence_compr(
        sentence_compression_instance,
        n=n_part_pos_examples
    )

    # Add dataset entry to chuck
    dataset_entry = {
        ID: idx,
        REFERENCE_EXAMPLE: caption,
        POS_EXAMPLES: positive_examples,
        PART_POS_EXAMPLES: part_pos_examples
    }
    dataset_chunk[INSTANCES].append(dataset_entry)

  return dataset_chunk


def create_dataset_sentence_compression(root: str,
                                        dataset,
                                        dataset_name: str = "sent_compr",
                                        split: str = TRAIN_SENT_COMPR,
                                        start_chunk: int = 0,
                                        max_chunk: Optional[int] = None,
                                        chunk_dim: int = CHUNK_DIM,
                                        n_pos_examples: int = N_EXAMPLES,
                                        n_part_pos_examples: int = N_PART_EXAMPLES,
                                        log_tqdm: bool = False) -> int:
  # Create dataset directory if it doesn't exist
  root = os.path.join(root, dataset_name)
  os.makedirs(root, exist_ok=True)

  # Calculate max chunk
  chunk_num = math.ceil(len(dataset[split]) / chunk_dim)
  chunk_num = chunk_num if max_chunk is None else min(chunk_num, max_chunk)

  # For each chunk
  iterable = range(start_chunk, chunk_num)
  iterable = tqdm(iterable, "Creating chunks...") if log_tqdm else iterable
  for chunk_idx in iterable:

    try:
      # Create the chunk
      dataset_chunk = create_dataset_chunck_sentence_compression(
          dataset=dataset,
          split=split,
          dataset_name=dataset_name,
          start_idx=chunk_idx*chunk_dim,
          n_pos_examples=n_pos_examples,
          n_part_pos_examples=n_part_pos_examples
      )

      # Store the chunk
      chunk_filename = os.path.join(root, f"{CHUNK_NAME}_{chunk_idx}.json")
      with open(chunk_filename, "w") as fp:
        json.dump(dataset_chunk, fp)

    except Exception as e:
      print(f"Chunk {chunk_idx} creation failed due to error: {e}.")
      print(f"Traceback: {traceback.format_exc()}")
      return chunk_idx - 1

  return chunk_idx


['A woman wearing a strange hat is also trance taking a picture, sitting on her bike in a sorry parking lot surrounded by trees.', 'A woman wearing a strangely strange hat is taken by taking a terrifying photo, sitting on her bike in a parking lot surrounded by trees.', 'A woman wearing a huge strange hat represents taking a picture, sitting on her motorcycle in a parking lot surrounded by large trees.', 'A woman obviously wearing a foreign crust hat is exactly an exhibition, sitting on her bike outside in a parking lot surrounded by trees.', 'A housekeeper wearing a strange hat necklace gets taken for a haunting photo, sitting on her bike in a darkened parking lot surrounded by trees.']


In [ ]:
create_dataset_sentence_compression("./prova_sent_compr",
                                    dataset_name="sent_compr",
                                    split=TRAIN_SENT_COMPR,
                                    dataset=sentence_compression_ds,
                                    max_chunk=10,
                                    log_tqdm=True)

Creating chunks...: 100%|██████████| 10/10 [30:08<00:00, 180.85s/it]


9

In [24]:
compute_cosine_scores(caption='Serge Ibaka -- the Oklahoma City Thunder forward who was born in the Congo but played in Spain -- has been granted Spanish citizenship and will play for the country in EuroBasket this summer, the event where spots in the 2012 Olympics will be decided.',
                      captions=['Serge Ibaka -- the Oklahoma City Thunder forward who was born in the Congo but played in Spain -- has been granted Spanish citizenship and will play for the country in EuroBasket this summer, the event where spots in the 2012 Olympics will be decided.', 'Serge Ibaka - - now the forward Oklahoma City Thunder who was born in Congo but played professionally in Espana - - the experience gained permanent Spanish citizenship and will play for the country at EuroBasket this summer, in case spots at the 2012 Summer Olympics will be decided.', 'Serge Serge Ibaka - - the forward Oklahoma City Thunder who was born in Congo but played in Spain - - has already officially granted Spanish citizenship and will play for the country to EuroBasket this summer, the event where the spots in the computer program of the 2012 Olympic Games will be decided.', 'Serge di Ibaka - - the forward Oklahoma City Thunder which was apparently born in Congo but played in Espana - - was given Spanish citizenship and will play for the country exclusively to EuroBasket this summer, the event where the spots at the 2012 Olympic Games will be decided.', 'Serge Ibaka - - the forward Oklahoma City Thunder which has been digested in part in the former Congo but which has already played in Spain - - has obtained Spanish citizenship and will play for the country at EuroBasket this summer, the event for which spots for the 2012 Olympic Games will be decided.'])

tensor([1.0000, 0.9780, 0.9752, 0.9714, 0.9783], device='cuda:0')

In [25]:
from torch.utils.data import DataLoader

print(os.getcwd())

qds = QuadrupletDataset("./prova_sent_compr/sent_compr",
                        *[f"chunk_{i}.json" for i in range(0, 10)],
                        hard_contrastive_mode = RANDOM,
                        n_pos=1,
                        n_neg=1,
                        n_part_pos=1,
                        cache_size=3)

# 1st chunk
print(qds[0])
print(qds[3])
print(qds[4])

# 2nd chunk
print(qds[qds.chunk_dim + 2])
print(qds[qds.chunk_dim + 4])

# 3rd chunk
print(qds[qds.chunk_dim*2])
print(qds[qds.chunk_dim*2 + 3])

# 5th chunk
print(qds[qds.chunk_dim*4 + 1])
print(qds[qds.chunk_dim*4 + 5])

# 6th chunk
print(qds[qds.chunk_dim*5 + 2])
print(qds[qds.chunk_dim*5 + 9])

# 3rd chunk
print(qds[qds.chunk_dim*2 + 2])
print(qds[qds.chunk_dim*2 + 5])

# 1st chunk
print(qds[6])
print(qds[2])

# Get more than one sample
print(qds[[1, 2, 3]])

#train_dataloader = DataLoader(training_data, batch_size=2, shuffle=True)

/media/neurone-pc3/a111f114-c06b-400b-8db7-4ce2466e2265/Andrea/sentence-transformer-proj/CoCo
{'id': 0, 'reference': 'Serge Ibaka -- the Oklahoma City Thunder forward who was born in the Congo but played in Spain -- has been granted Spanish citizenship and will play for the country in EuroBasket this summer, the event where spots in the 2012 Olympics will be decided.', 'positive': 'Serge Ibaka - - Oklahoma City Thunder before World Health Organization was born in Congo but played in Spain - - would have temporarily obtained Spanish citizenship and will play for the country at EuroBasket this summer, the event where UEFA fleck in winter 2012 Olympiad will be decided.', 'part_positive': 'The EuroBasket region this summer, the event where the situation in the 2012 Olympic Games remains to be decided.', 'negative': 'Massey Energy, the fourth largest coal exchange in the western region, could also have its basic charter cancelled if coppers of public interest made their way.'}
{'id': 3, 're

In [ ]:
from sentence_transformers import SentenceTransformer
from typing import Tuple, Any


class QuadrupletSentenceTransformerLoss(torch.nn.Module):
  def __init__(self,
               st_model: SentenceTransformer,
               quadruplet_loss: QuadrupletLoss,
               additional_model_kwargs: Optional[List[str]] = None,
               additional_loss_kwargs: Optional[List[str]] = None):
    super().__init__()
    self._st_model = st_model
    self._quadruplet_loss = quadruplet_loss
    self.__additional_model_kwargs = additional_model_kwargs
    self.__additional_loss_kwargs = additional_loss_kwargs
    pass

  def forward(self, features: dict, labels = None) -> Tuple[torch.Tensor, ...]:
    reference_example = features[REFERENCE_EXAMPLE]
    pos_example = features[POS_EXAMPLES]
    part_pos_example = features[PART_POS_EXAMPLES]
    neg_example = features[NEG_EXAMPLES]

    # Additional model arguments
    additional_model_kwargs = {}
    for kwarg in self.__additional_model_kwargs:
      additional_model_kwargs[kwarg] = features[kwarg]

    # Call the model on the instance examples
    reference_example = self._st_model(
        reference_example,
        **additional_model_kwargs
    )

    pos_example = self._st_model(
        pos_example,
        **additional_model_kwargs
    )

    part_pos_example = self._st_model(
        part_pos_example,
        **additional_model_kwargs
    )

    neg_example = self._st_model(
        neg_example,
        **additional_model_kwargs
    )

    # Additional loss kwargs
    additional_loss_kwargs = {}
    for kwarg in self.__additional_loss_kwargs:
      additional_loss_kwargs[kwarg] = features[kwarg]

    # Compute loss
    loss = self._quadruplet_loss(
        x_anchor=reference_example,
        x_pos=pos_example,
        x_part_pos=part_pos_example,
        x_neg=neg_example,
        **additional_loss_kwargs
    )

    return loss

# Ensures compatbility with SentenceTransformer's fit() expecting a label
def add_empty_label(instance: Any) -> Tuple[Any, str]:
  return instance, ""
